# AFM

在 Ebedding 之后加入了注意力机制的池化层。

In [1]:
# build train data

import os
import numpy as np

BASEDIR = os.getcwd()

fields_dict = {}
lines = None

with open(os.path.join(BASEDIR, 'assets/datasets/criteo_ctr/small_train.txt')) as f:
    lines = f.readlines()

fields_dict = {}
for line in lines:
    line = line.strip('\n')

    for elem in line.split(' ')[1:]:
        field, feature, _ = elem.split(':')

        if field not in fields_dict:
            fields_dict[field] = {'index': len(fields_dict), 'features': {}, 'last_idx': -1}

        if feature not in fields_dict[field]['features']:
            fields_dict[field]['features'][feature] = fields_dict[field]['last_idx'] + 1
            fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1

for field in fields_dict.keys():
    if 'none' not in fields_dict[field]['features']:
        fields_dict[field]['features']['none'] = fields_dict[field]['last_idx'] + 1
        fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1


def init_field_tensor(fields_dict):
    init_tensor = np.zeros((len(fields_dict), 1))
    for field in fields_dict.keys():
        init_tensor[fields_dict[field]['index']] = fields_dict[field]['last_idx']
    return init_tensor.astype(int)


X_train = []
y_train = []

for line in lines:
    line = line.strip('\n')
    elems = line.split(' ')
    y_train.append(float(elems[0]))

    init_tensors = init_field_tensor(fields_dict)
    for elem in elems[1:]:
        field, feature, _ = elem.split(':')
        field_idx = fields_dict[field]['index']
        feature_idx = fields_dict[field]['features'][feature]
        init_tensors[field_idx] = feature_idx
    X_train.append(init_tensors)

X_train = np.concatenate(X_train, 1)
y_train = np.array(y_train)

In [2]:
# build embedding layer

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

class FieldEmbeddingBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FieldEmbeddingBlock, self).__init__()
        self.fields_dict = fields_dict
        self.fields_embedding = {}
        self.embedding_size = embedding_size

        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            self.fields_embedding[field_idx] = nn.Embedding(len(self.fields_dict[field]['features']),
                                                            self.embedding_size)

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[0] * self.embedding_size, input_field_tensor.shape[1])
        idx = 0
        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            out[idx:idx + self.embedding_size, :] = self.fields_embedding[field_idx](input_field_tensor[field_idx, :])
            idx += self.embedding_size
        return out.double()


class CrossBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(CrossBlock, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size
        self.output_dimension = self.field_cnt ** 2

    def forward(self, x):
        out = torch.zeros(self.output_dimension, x.shape[1])
        idx = 0
        for field1 in self.fields_dict.keys():
            for field2 in self.fields_dict.keys():
                field1_tensor_idx = self.fields_dict[field1]['index']
                field2_tensor_idx = self.fields_dict[field2]['index']
                out[idx, :] = x[field1_tensor_idx:field1_tensor_idx + self.embedding_size,
                                                        :].mul(
                    x[field2_tensor_idx:field2_tensor_idx + self.embedding_size, :]).sum(0)
                idx += 1
        return out.double()


class AttentionBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size, t):
        super(AttentionBlock, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size
        self.t = t
        self.h = torch.nn.Parameter(torch.randn(self.field_cnt * self.field_cnt, self.t, dtype=torch.double))
        self.activate = nn.Sequential(
            nn.Linear(self.field_cnt ** 2, self.t).double(),
            nn.ReLU()
        )

    def forward(self, x):
        out = self.activate(x.T).T
        h_prime = self.h.mm(out)
        h_norm = F.softmax(h_prime, dim=0)
        return h_norm


class AFM(nn.Module):
    def __init__(self, fields_dict, embedding_size, t):
        super(AFM, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size
        self.t = t
        self.embedding_block = FieldEmbeddingBlock(fields_dict, embedding_size)
        self.cross_block = CrossBlock(fields_dict, embedding_size)
        self.attention_block = AttentionBlock(fields_dict, embedding_size, self.t)
        self.out_layer = nn.Linear(self.field_cnt ** 2, 1, bias=True).double()

    def forward(self, x):
        out = self.embedding_block(x)
        out1 = self.cross_block(out)
        attentions = self.attention_block(out1)
        out = out1.mul(attentions)
        out = self.out_layer(out.T)
        return F.sigmoid(out)

In [3]:
# PyTorch Version

import torch.optim as optim

device = torch.device('cpu')
LEARNING_RATE = 1e-3

EPOCH = 100
PRINT_STEP = EPOCH / 10
N = len(y_train)
t = 10

HIDDEN_DIMENSION = 8
EMBEDDING_SIZE = 8
afm = AFM(fields_dict, EMBEDDING_SIZE, 10)

BATCH_SIZE = 8
loss_fn = nn.BCELoss()
optimizer = optim.Adam(afm.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):

    index = np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)
    X_batch = torch.from_numpy(X_train[:, index]).long()
    y_batch = torch.from_numpy(y_train[index]).reshape(-1, BATCH_SIZE)

    y_hat = afm(X_batch).reshape(-1, BATCH_SIZE)
    loss = loss_fn(y_hat, y_batch)

    loss.backward()
    optimizer.step()

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, loss))


EPOCH: 0, loss: 0.685074
EPOCH: 10, loss: 0.688310
EPOCH: 20, loss: 0.722300
EPOCH: 30, loss: 0.645165
EPOCH: 40, loss: 0.590204
EPOCH: 50, loss: 0.632628
EPOCH: 60, loss: 0.607923
EPOCH: 70, loss: 0.829914
EPOCH: 80, loss: 0.586729
EPOCH: 90, loss: 0.449997
